In [1]:
# BERT
!pip install transformers==4.15.0 sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 69.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=083cbd8d8268f079b5e2d9878d6f61a909d378a39aea428264a38b34dae3df17
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [2]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification, AutoTokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD
import pickle

# Read CSV Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pickle.load(open('drive/MyDrive/AIBuilders/tpth/ner_ds_40k_nova.pkl', 'rb'))
df.tail()

,text,labels
42885,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
42886,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
42887,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(1), tensor(1), t..."
42888,"{'input_ids': [[tensor(5), tensor(13276), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
42889,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(1), tensor(1), t..."


In [5]:
df_test = df[37890:]
df = df[:37890]
df_train, df_val = np.split(df.sample(frac=1, random_state=42),
                            [int(.9 * len(df))])

In [ ]:
print(len(df_train), len(df_val))

34101 3789


In [ ]:
df.iloc[0]['labels']

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    1,    1,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -1

# Initialize Tokenizer

In [6]:
tokenizer_th = pickle.load(open('drive/MyDrive/AIBuilders/tpth/tokenizer_40k_nova.pkl', 'rb'))

# Create Dataset Class

In [7]:
class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        # lb = [i.split() for i in df['labels'].values.tolist()]
        # txt = df['text'].values.tolist()
        # self.texts = [tokenizer_th(str(i),
        #                        padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        # self.labels = [align_label(i,j) for i,j in zip(txt, lb)]
        self.texts = [df.iloc[i]['text'] for i in range(df.shape[0])]
        self.labels = [df.iloc[i]['labels'] for i in range(df.shape[0])]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels


# Split Data

In [ ]:
asdf = DataSequence(df_train)
asdf.__getitem__(1)

({'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Build Model

In [8]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer_th))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [9]:
model = BertModel()
# model = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
# model.resize_token_embeddings(len(tokenizer_th))

# FILE = "drive/MyDrive/AIBuilders/tagging.pth"
# FILE = "drive/MyDrive/AIBuilders/tagging_nova_75.pth"
FILE = "drive/MyDrive/AIBuilders/tpth/tagging_tpth_200.pth"
model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.3.output.dense.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.9.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.6.attention.output.LayerNorm.bias', 'lm_head.decoder.weight', 'roberta.encoder.layer.5.output.LayerNorm.bias', 'lm_head.decoder.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.3.intermediate.dense.bias', 'roberta.encoder.layer.5.attention.self.value.weight', 'roberta.encoder.layer.8.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.intermediate.dense.bias', 'roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.3.output.dense.weight', 'roberta.encoder.layer.3.attention.self.value.weight', 'roberta.encoder.layer.5.output.LayerNorm.weight', 'roberta.emb

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
from huggingface_hub import notebook_login

notebook_login() # เอาไว้โยนโมเดลขึ้น hugging face ได้เลย

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import TrainingArguments

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"

batch_size = 32
# Show the training loss with every epoch
logging_steps = len(df_train) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-tagging",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    num_train_epochs = 75,
    fp16=True, # สำหรับคนใช้ GPU
    logging_steps=logging_steps,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train,
    eval_dataset=df_test,
)

/content/wangchanberta-base-att-spm-uncased-tagging is already a clone of https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-tagging. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wangchanberta-base-att-spm-uncased-tagging
Configuration saved in wangchanberta-base-att-spm-uncased-tagging/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-tagging/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.34k/425M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.92k/2.92k [00:00<?, ?B/s]

To https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-tagging
   3c3ed19..daa973a  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}}
To https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-tagging
   daa973a..712de4d  main -> main



'https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-tagging/commit/daa973a5e4f9d4068f6ffab57f3a3af8d9f1b64a'

In [ ]:
model

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

# Model Training

In [ ]:
BATCH_SIZE = 32

def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000
    
    for epoch_num in range(EPOCHS):
        total_acc_train = 0
        total_loss_train = 0

        model.train()
        
        for train_data, train_label in tqdm(train_dataloader):
            train_label = train_label[0].to(device)
            mask = train_data['attention_mask'][0].to(device)
            input_id = train_data['input_ids'][0].to(device)
            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            logits_clean = logits[0][train_label != -100]
            label_clean = train_label[train_label != -100]
            predictions = logits_clean.argmax(dim=1)
            pred_len = len(predictions)
            for i in range(pred_len):
                if predictions[i] == 0:
                    predictions[i] = -100
            
            # print(predictions)
            # print(label_clean)
            numer = 0
            denom = 0
            for i in range(pred_len):
                if label_clean[i] == 1:
                    denom += 1
                    if predictions[i] == 1:
                        numer += 1
                elif label_clean[i] == 0 and predictions[i] == 1:
                    denom += 1

            if denom == 0:
              acc = 0
            else:
              acc = float(numer)/float(denom)
            # print(f"train acc: {acc}")
            total_acc_train += acc * BATCH_SIZE
            total_loss_train += loss.item() * BATCH_SIZE

            loss.backward()
            optimizer.step()
        
        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label[0].to(device)
            mask = val_data['attention_mask'][0].to(device)

            input_id = val_data['input_ids'][0].to(device)

            loss, logits = model(input_id, mask, val_label)

            logits_clean = logits[0][val_label != -100]
            label_clean = val_label[val_label != -100]
            predictions = logits_clean.argmax(dim=1)   
            pred_len = len(predictions)
            for i in range(pred_len):
                if predictions[i] == 0:
                    predictions[i] = -100       

            numer = 0
            denom = 0
            for i in range(pred_len):
                if label_clean[i] == 1:
                    denom += 1
                    if predictions[i] == 1:
                        numer += 1
                elif label_clean[i] == 0 and predictions[i] == 1:
                    denom += 1
        
            if denom == 0:
              acc = 0
            else:
              acc = float(numer)/float(denom)
            # print(f"valid acc: {acc}")
            total_acc_val += acc * BATCH_SIZE
            total_loss_val += loss.item() * BATCH_SIZE

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 1e-3
EPOCHS = 100



In [ ]:
train_loop(model, df_train, df_val)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 1066/1066 [02:09<00:00,  8.21it/s]


Epochs: 1 | Loss:  0.132 | Accuracy:  0.454 | Val_Loss:  0.138 | Accuracy:  0.479


100%|██████████| 1066/1066 [02:10<00:00,  8.18it/s]


Epochs: 2 | Loss:  0.134 | Accuracy:  0.459 | Val_Loss:  0.136 | Accuracy:  0.471


100%|██████████| 1066/1066 [02:09<00:00,  8.23it/s]


...

Epochs: 98 | Loss:  0.124 | Accuracy:  0.482 | Val_Loss:  0.124 | Accuracy:  0.513


100%|██████████| 1066/1066 [02:12<00:00,  8.04it/s]


Epochs: 99 | Loss:  0.124 | Accuracy:  0.495 | Val_Loss:  0.130 | Accuracy:  0.515


100%|██████████| 1066/1066 [02:12<00:00,  8.02it/s]


Epochs: 100 | Loss:  0.124 | Accuracy:  0.490 | Val_Loss:  0.124 | Accuracy:  0.515


In [ ]:
FILE = "drive/MyDrive/AIBuilders/tagging_tpth_100.pth"
torch.save(model.state_dict(), FILE)

In [ ]:
train_loop(model, df_train, df_val)

In [ ]:
FILE = "drive/MyDrive/AIBuilders/tagging_tpth_200.pth"
torch.save(model.state_dict(), FILE)

# Evaluate Model

In [11]:
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    cou = 0
    total_precision = [0,0]
    total_recall = [0,0]
    total_f1 = [0,0]
    for test_data, test_label in test_dataloader:
        # if cou == 20:
        #     break
        cou+=1
        if cou%100==0:
          print(f"============={cou}=================")

        test_label = test_label[0].to(device)
        mask = test_data['attention_mask'][0].to(device)
        input_id = test_data['input_ids'][0].to(device)
          
        loss, logits = model(input_id, mask, test_label.long())
        # print(test_data)
        # print(test_label)
        # print(loss)
        # print(logits)

        logits_clean = logits[0][test_label != -100]
        label_clean = test_label[test_label != -100]
        predictions = logits_clean.argmax(dim=1)
        pred_len = len(predictions)
        for i in range(pred_len):
            if predictions[i] == 0:
                predictions[i] = -100
        # print(logits_clean)
        # print(label_clean)
        # print(predictions)

  
        a = tokenizer_th.convert_ids_to_tokens(input_id.squeeze())
        # print(f"input: {''.join(a)}")
        # print(label_clean)
        # print(len(input_id[0]), len(label_clean))
        # print("________________________")

        ids_to_labels = {0:'f', 1:'i', -100:'f'}
        prediction_label = [ids_to_labels[i] for i in predictions.tolist()]
        # print("PREDICTIONS")
        # for i in range(len(predictions)):
        #   if prediction_label[i] == 'i':
        #     print(a[i])
        # print("LABELS")
        # for i in range(len(label_clean)):
        #   if label_clean[i] == 1:
        #     print(a[i])   

        # acc = (predictions == label_clean).float().mean()

        # print(predictions, label_clean)
        TP = [0,0]
        FP = [0,0]
        TN = [0,0]
        FN = [0,0]
        for i in range(pred_len):
          if predictions[i] == 1 and label_clean[i] == 1:
            TP[1] += 1
            TN[0] += 1
          elif predictions[i] == -100 and label_clean[i] == 1:
            FP[0] += 1
            FN[1] += 1
          elif predictions[i] == 1 and label_clean[i] == 0:
            FP[1] += 1
            FN[0] += 1
          elif predictions[i] == -100 and label_clean[i] == 0:
            TP[0] += 1
            TN[1] += 1

        precision = [float(TP[0]) / float(TP[0]+FP[0]) if TP[0]+FP[0] > 0 else 0, float(TP[1]) / float(TP[1]+FP[1]) if TP[1]+FP[1] > 0 else 0]
        recall = [float(TP[0]) / float(TP[0]+FN[0]) if TP[0]+FN[0] > 0 else 0, float(TP[1]) / float(TP[1]+FN[1]) if TP[1]+FN[1] > 0 else 0]
        f1 = [float(2*precision[0]*recall[0]) / float(precision[0] + recall[0]) if precision[0]+recall[0] > 0 else 0, float(2*precision[1]*recall[1]) / float(precision[1] + recall[1]) if precision[1]+recall[1] > 0 else 0]
        # print(f"0 >>> TP:{TP[0]}   TN:{TN[0]}   FP:{FP[0]}   FN:{FN[0]}   precision:{precision[0]}    recall:{recall[0]}    f1:{f1[0]}")
        # print(f"1 >>> TP:{TP[1]}   TN:{TN[1]}   FP:{FP[1]}   FN:{FN[1]}   precision:{precision[1]}    recall:{recall[1]}    f1:{f1[1]}")
        total_precision[0] += precision[0]
        total_precision[1] += precision[1]
        total_recall[0] += recall[0]
        total_recall[1] += recall[1]
        total_f1[0] += f1[0]
        total_f1[1] += f1[1]

        numer = 0
        denom = 0
        for i in range(pred_len):
            if label_clean[i] == 1:
                denom += 1
                if predictions[i] == 1:
                    numer += 1
            elif label_clean[i] == 0 and predictions[i] == 1:
                denom += 1
        
        acc = float(numer)/float(denom) if denom > 0 else 0
        # print(f"ACC: {acc}")
        # print("-----------------------------------")
        total_acc_test += acc

    val_accuracy = total_acc_test / len(df_test)
    # print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')
    print(f'Test Accuracy: {total_acc_test / cou: .3f}')
    print(f'0 >>> precision: {total_precision[0] / cou: .3f}    recall: {total_recall[0] / cou: .3f}    f1: {total_f1[0] / cou: .3f}')
    print(f'1 >>> precision: {total_precision[1] / cou: .3f}    recall: {total_recall[1] / cou: .3f}    f1: {total_f1[1] / cou: .3f}')

evaluate(model, df_test)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


=============100=================
=============200=================
=============300=================
=============400=================
=============500=================
=============600=================
=============700=================
=============800=================
=============900=================
=============1000=================
=============1100=================
=============1200=================
=============1300=================
=============1400=================
=============1500=================
=============1600=================
=============1700=================
=============1800=================
=============1900=================
=============2000=================
=============2100=================
=============2200=================
=============2300=================
=============2400=================
=============2500=================
=============2600=================
=============2700=================
=============2800=================
=============2900============

# Predict One Sentence

In [ ]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer_th(texts, padding='max_length', max_length=512, truncation=True)
    c = tokenizer_th.convert_ids_to_tokens(tokenized_inputs.input_ids)
    print(f"ids_to_tokens: {c}")

    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        else:
            try:
              label_ids.append(2)
            except:
                label_ids.append(-100)

        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer_th(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'][0].unsqueeze(0).to(device)
   
    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)
   
    logits = model(input_id, mask, None)
    # print(input_id)
    # print(mask)
    # print(label_ids)
    logits_clean = logits[0][label_ids != -100]
    # print(logits_clean)
    # print(len(logits[0][0]), len(logits_clean))

    predictions = logits_clean.argmax(dim=1).tolist()
    ids_to_labels = {0:'f', 1:'i'}
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(prediction_label)

    tokenized_inputs = tokenizer_th(sentence, padding='max_length', max_length=512, truncation=True)
    c = tokenizer_th.convert_ids_to_tokens(tokenized_inputs.input_ids)
    print(c)
    for i, j in enumerate(prediction_label):
        if j == 'i':
            print(c[i], c[i+1], c[i+2])
            
evaluate_one_text(model, 'ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ')

ids_to_tokens: ['<s>', '▁', 'ประเทศ', 'เรา', 'ผลิต', 'และ', 'ส่งออก', 'ยา', '▁', 'สูบ', 'เยอะ', 'สุดในโลก', 'จิง', 'ป่าว', 'คับ', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

In [ ]:
FILE = "tagging.pth"
torch.save(model.bert.state_dict(), FILE)